In [18]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [19]:
df = pd.read_csv("C:/Users/Noir/Desktop/data/bank-additional-full.csv", sep=";")
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [20]:
df.shape

(41188, 21)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [22]:
df["y"].unique()

array(['no', 'yes'], dtype=object)

In [23]:
df.loc[df["y"]=="yes", "y"] = 1
df.loc[df["y"]=="no", "y"] = 0
df["y"] = df["y"].astype(int)
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


In [24]:
train_all, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train_all, test_size=0.2)
print(len(train), '训练集数目')
print(len(val), '验证集数目')
print(len(test), '测试集数目')

26360 训练集数目
6590 验证集数目
8238 测试集数目


In [25]:
def df_to_dataset(df, shuffle=True, batch_size=32):
    labels = df.pop('y')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [26]:
batch_size = 32
train_ds = df_to_dataset(train, shuffle=True, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [27]:
# 分类列名称列表
category_names = ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "day_of_week", "poutcome"]

# 数值列名称列表
numeric_names = ["age", "duration", "campaign", "pdays", "previous"]

In [28]:

# 数据统计，分类列的枚举列表
category_vocabulary_list = {}

for feature_name in category_names:
    category_vocabulary_list[feature_name] = list(df[feature_name].unique())

print(category_vocabulary_list)

{'job': ['housemaid', 'services', 'admin.', 'blue-collar', 'technician', 'retired', 'management', 'unemployed', 'self-employed', 'unknown', 'entrepreneur', 'student'], 'marital': ['married', 'single', 'divorced', 'unknown'], 'education': ['basic.4y', 'high.school', 'basic.6y', 'basic.9y', 'professional.course', 'unknown', 'university.degree', 'illiterate'], 'default': ['no', 'unknown', 'yes'], 'housing': ['no', 'yes', 'unknown'], 'loan': ['no', 'yes', 'unknown'], 'contact': ['telephone', 'cellular'], 'month': ['may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'mar', 'apr', 'sep'], 'day_of_week': ['mon', 'tue', 'wed', 'thu', 'fri'], 'poutcome': ['nonexistent', 'failure', 'success']}


In [29]:
feature_columns = []

for feature_name in category_names:
    feature_columns.append(
        feature_column.indicator_column(
            feature_column.categorical_column_with_vocabulary_list(feature_name, category_vocabulary_list[feature_name])
        )
    )
    
for feature_name in numeric_names:
    feature_columns.append(feature_column.numeric_column(feature_name))

feature_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='job', vocabulary_list=('housemaid', 'services', 'admin.', 'blue-collar', 'technician', 'retired', 'management', 'unemployed', 'self-employed', 'unknown', 'entrepreneur', 'student'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='marital', vocabulary_list=('married', 'single', 'divorced', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='education', vocabulary_list=('basic.4y', 'high.school', 'basic.6y', 'basic.9y', 'professional.course', 'unknown', 'university.degree', 'illiterate'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='default', vocabulary_list=('no', 'unknown', 'yes'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical

In [30]:
model = tf.keras.Sequential([
    layers.DenseFeatures(feature_columns),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
818/824 [============================>.] - ETA: 0s - loss: 0.8901 - accuracy: 0.8765WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'job': <tf.Tensor 'ExpandDims_12:0' shape=(None, 1) dtype=string>, 'marital': <tf.Tensor 'ExpandDims_14:0' shape=(None, 1) dtype=string>, 'education': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=string>, 'default': <tf.Tensor 'ExpandDims_6:0' shape=(None, 1) dtype=string>, 'housing': <tf.Tensor 'ExpandDims_11:0' shape=(None, 1) dtype=string>, 'loan': <tf.Tensor 'ExpandDims_13:0' shape=(None, 1) dtype=string>, 'contact': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=string>, 'month': <tf.Tensor 'ExpandDims_15:0' shape=(None, 1) dtype=string>, 'day_of_week': <tf.Tensor 'ExpandDims_5:0' sha

In [31]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

258/258 [==============================] - 0s 1ms/step - loss: 0.2129 - accuracy: 0.9033
Accuracy 0.9032531976699829
